In [1]:
import pandas as pd
import itertools
from pandas.tseries.holiday import USFederalHolidayCalendar

## README

### Stock price
- Adjusted for stock splits and dividend
- Check every trading day
    - Some trading days are missing 
    - 3 days (2021-04-02, 2022-04-15, 2022-06-20)
- Check every trading 5-minute from 4am-8pm 
    - Percentage of missing 5-minute per day
    - For days with low liquidity (e.g. Christimas), we have limited trading activity and price data
- Forward fill missing minute
    - Set maximum limit of FORWARD_FILL_LIMIT x 5-minute interval to forward fill
- Calculate percentage price return in x-minutes
    - RETURN_WINDOW x 5-minute interval

### News
- Round up to 5-minute level

### Stock price - News 
- Merge stock price with news    
    - Check % of news dropped (outside trading hours or no data)


### Todo
- Should we include outside trading hours?

In [2]:
# config

# unit is 5-minute
FORWARD_FILL_LIMIT = 5
RETURN_WINDOW = 5

In [3]:
# v1
# stock_df = pd.read_csv("./stock_price/stock_price_v1/stock_price_2019.csv", 
#                        parse_dates=["datetime", "datetime_us_eastern", "datetime_utc", "date", "time"])

# v2
stock_df = pd.read_csv("./stock_price/stock_price_v2/stock_price_v2.csv", 
                       parse_dates=["time"])
stock_df.rename(columns={"time":"datetime"}, inplace=True)
stock_df.drop_duplicates(inplace=True)

# check no duplicate datetime per symbol
assert stock_df.groupby("symbol").apply(lambda x:x["datetime"].duplicated().any()).any() == False

# create date and time columns
stock_df["date"] = stock_df["datetime"].dt.normalize()
stock_df["time"] = stock_df["datetime"].dt.time

In [4]:
stock_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22009293 entries, 0 to 22009539
Data columns (total 9 columns):
 #   Column    Dtype         
---  ------    -----         
 0   datetime  datetime64[ns]
 1   open      float64       
 2   high      float64       
 3   low       float64       
 4   close     float64       
 5   volume    int64         
 6   symbol    object        
 7   date      datetime64[ns]
 8   time      object        
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 1.6+ GB


In [5]:
stock_df

,datetime,open,high,low,close,volume,symbol,date,time
0,2023-03-28 20:00:00,53.8100,53.8800,53.81,53.8800,3375,SCHW,2023-03-28,20:00:00
1,2023-03-28 19:55:00,53.8500,53.8500,53.84,53.8400,1804,SCHW,2023-03-28,19:55:00
2,2023-03-28 19:50:00,53.8400,53.8800,53.84,53.8800,3898,SCHW,2023-03-28,19:50:00
3,2023-03-28 19:45:00,53.8500,53.8500,53.84,53.8400,802,SCHW,2023-03-28,19:45:00
4,2023-03-28 19:40:00,53.8499,53.8499,53.82,53.8200,1928,SCHW,2023-03-28,19:40:00
...,...,...,...,...,...,...,...,...,...
22009535,2021-04-08 09:40:00,130.5500,130.6200,130.10,130.2300,19929,TMUS,2021-04-08,09:40:00
22009536,2021-04-08 09:35:00,130.8800,130.9063,130.40,130.6161,73680,TMUS,2021-04-08,09:35:00
22009537,2021-04-08 09:30:00,130.8300,130.8400,130.83,130.8400,1001,TMUS,2021-04-08,09:30:00
22009538,2021-04-08 09:15:00,130.9900,130.9900,130.98,130.9800,601,TMUS,2021-04-08,09:15:00


In [6]:
stock_df.groupby("symbol").agg({"date":["min", "max"]})

date           
              min        max
symbol                      
A      2021-04-08 2023-03-28
AAL    2021-04-09 2023-03-29
AAP    2021-04-09 2023-03-29
AAPL   2021-04-08 2023-03-28
ABBV   2021-04-08 2023-03-28
...           ...        ...
YUM    2021-04-08 2023-03-28
ZBH    2021-04-08 2023-03-28
ZBRA   2021-04-09 2023-03-29
ZION   2021-04-09 2023-03-29
ZTS    2021-04-08 2023-03-28

[503 rows x 2 columns]

## Check trading day 

In [7]:
actual_df = stock_df[["symbol", "date"]].drop_duplicates().reset_index(drop=True)

In [8]:
tickers = stock_df["symbol"].unique()
trading_dates = pd.bdate_range(start="2021-04-09", 
                               end="2023-03-28", 
                               freq='C', 
                               holidays=USFederalHolidayCalendar().holidays().tolist()
                              )

expected_df = pd.DataFrame([x for x in itertools.product(tickers, trading_dates)], columns=["symbol", "date"])

In [9]:
combined_df = expected_df.merge(actual_df.assign(actual=1), how="left", on=["symbol", "date"], validate="one_to_one")

def count_missing_dates(df):
    missing_dates = df["date"].dt.date.tolist()
    return pd.Series([missing_dates, len(missing_dates)])
    
missing_dates_df = combined_df.loc[(combined_df.isna().any(axis=1))].groupby("symbol").apply(count_missing_dates)
missing_dates_df.columns = ["missing_dates", "count"]

In [10]:
missing_dates_df.loc[missing_dates_df["count"]>2]

,missing_dates,count
symbol,,
BALL,"[2021-04-09, 2021-04-12, 2021-04-13, 2021-04-1...",273
BBWI,"[2021-04-09, 2021-04-12, 2021-04-13, 2021-04-1...",82
CEG,"[2021-04-09, 2021-04-12, 2021-04-13, 2021-04-1...",206
CTRA,"[2021-04-09, 2021-04-12, 2021-04-13, 2021-04-1...",125
ELV,"[2021-04-09, 2021-04-12, 2021-04-13, 2021-04-1...",306
GEHC,"[2021-04-09, 2021-04-12, 2021-04-13, 2021-04-1...",435
GEN,"[2021-04-09, 2021-04-12, 2021-04-13, 2021-04-1...",398
META,"[2021-04-09, 2021-04-12, 2021-04-13, 2021-04-1...",149
OGN,"[2021-04-09, 2021-04-12, 2021-04-13, 2021-04-1...",40


## Check trading hours

In [11]:
date = '2021-04-09'
expected_minutes = pd.date_range(start=f'{date} 04:00:00', end=f'{date} 20:00:00', freq='5T').time

assert len(expected_minutes) == (12 * 16) + 1

In [12]:
actual_minutes_df = stock_df.groupby(["symbol", "date"]).agg({"time":["min", "max", "count"]})
#  remove first level column index
actual_minutes_df.columns = actual_minutes_df.columns.get_level_values(1)
actual_minutes_df["count_perc"] = actual_minutes_df["count"] / len(expected_minutes)

In [13]:
# check daily proportion of missing minutes 
actual_minutes_df.groupby("symbol").agg({"count_perc":["mean", "min", "max"]})

count_perc                    
             mean       min       max
symbol                               
A        0.420075  0.238342  0.616580
AAL      0.862262  0.424870  0.994819
AAP      0.422566  0.238342  0.730570
AAPL     0.971404  0.621762  0.994819
ABBV     0.495408  0.248705  0.720207
...           ...       ...       ...
YUM      0.420304  0.238342  0.502591
ZBH      0.416176  0.233161  0.647668
ZBRA     0.401685  0.222798  0.461140
ZION     0.423734  0.222798  0.911917
ZTS      0.420283  0.233161  0.528497

[503 rows x 3 columns]

In [7]:
def fill_missing_minutes(df, limit=FORWARD_FILL_LIMIT):
    """
    Forward fill missing minutes with a maximum limit of `limit`.
    """
    date = df["date"].iloc[0]
    index = pd.date_range(start=f'{date} 04:00:00', end=f'{date} 20:00:00', freq='5T')
    metrics_col = ["open", "high", "low", "close", "volume"]
    # forward fill maximum of FORWARD_FILL_LIMIT
    filled_df = df.set_index("datetime").sort_index()[metrics_col].reindex(index, method="ffill", limit=limit)
    filled_df.index.name = "datetime"
    return filled_df

filled_stock_df = stock_df.groupby(["symbol", "date"]).apply(fill_missing_minutes).reset_index().dropna()

In [8]:
print(f"Before forward fill: {len(stock_df):,}")
print(f"After forward fill: {len(filled_stock_df):,}")

filled_stock_df

Before forward fill: 22,009,293
After forward fill: 26,173,189


,symbol,date,datetime,open,high,low,close,volume
55,A,2021-04-08,2021-04-08 08:35:00,128.655573,128.655573,128.655573,128.655573,700.0
56,A,2021-04-08,2021-04-08 08:40:00,128.655573,128.655573,128.655573,128.655573,1642.0
57,A,2021-04-08,2021-04-08 08:45:00,128.655573,128.655573,128.655573,128.655573,1642.0
58,A,2021-04-08,2021-04-08 08:50:00,128.655573,128.655573,128.655573,128.655573,1642.0
59,A,2021-04-08,2021-04-08 08:55:00,128.655573,128.655573,128.655573,128.655573,1642.0
...,...,...,...,...,...,...,...,...
47735994,ZTS,2023-03-28,2023-03-28 16:10:00,165.040000,165.040000,165.040000,165.040000,51999.0
47735995,ZTS,2023-03-28,2023-03-28 16:15:00,165.040000,165.040000,165.040000,165.040000,51999.0
47735996,ZTS,2023-03-28,2023-03-28 16:20:00,165.040000,165.040000,165.040000,165.040000,51999.0
47735997,ZTS,2023-03-28,2023-03-28 16:25:00,165.040000,165.040000,165.040000,165.040000,51999.0


In [16]:
stock_df.loc[(stock_df["symbol"] == "A") & (stock_df["date"] == "2021-04-08")].sort_values(by="datetime").head(20)

,datetime,open,high,low,close,volume,symbol,date,time
4049586,2021-04-08 08:35:00,128.655573,128.655573,128.655573,128.655573,700,A,2021-04-08,08:35:00
4049585,2021-04-08 08:40:00,128.655573,128.655573,128.655573,128.655573,1642,A,2021-04-08,08:40:00
4049584,2021-04-08 09:25:00,128.655573,128.655573,128.655573,128.655573,2618,A,2021-04-08,09:25:00
4049583,2021-04-08 09:35:00,128.912884,129.971819,128.530381,129.942129,86243,A,2021-04-08,09:35:00
4049582,2021-04-08 09:40:00,129.942129,130.347889,129.793680,130.219233,26724,A,2021-04-08,09:40:00
4049581,2021-04-08 09:45:00,130.268716,130.367682,129.833266,129.892646,7491,A,2021-04-08,09:45:00
4049580,2021-04-08 09:50:00,129.922336,129.942129,129.773887,129.848111,6111,A,2021-04-08,09:50:00
4049579,2021-04-08 09:55:00,129.853060,130.229130,129.763990,129.763990,12472,A,2021-04-08,09:55:00
4049578,2021-04-08 10:00:00,129.833266,129.833266,129.318644,129.378024,7164,A,2021-04-08,10:00:00
4049577,2021-04-08 10:05:00,129.348334,129.437403,129.199885,129.308748,9938,A,2021-04-08,10:05:00


In [17]:
filled_stock_df.loc[(filled_stock_df["symbol"] == "A") & (filled_stock_df["date"] == "2021-04-08")].head(20)

,symbol,date,datetime,open,high,low,close,volume
55,A,2021-04-08,2021-04-08 08:35:00,128.655573,128.655573,128.655573,128.655573,700.0
56,A,2021-04-08,2021-04-08 08:40:00,128.655573,128.655573,128.655573,128.655573,1642.0
57,A,2021-04-08,2021-04-08 08:45:00,128.655573,128.655573,128.655573,128.655573,1642.0
58,A,2021-04-08,2021-04-08 08:50:00,128.655573,128.655573,128.655573,128.655573,1642.0
59,A,2021-04-08,2021-04-08 08:55:00,128.655573,128.655573,128.655573,128.655573,1642.0
60,A,2021-04-08,2021-04-08 09:00:00,128.655573,128.655573,128.655573,128.655573,1642.0
61,A,2021-04-08,2021-04-08 09:05:00,128.655573,128.655573,128.655573,128.655573,1642.0
65,A,2021-04-08,2021-04-08 09:25:00,128.655573,128.655573,128.655573,128.655573,2618.0
66,A,2021-04-08,2021-04-08 09:30:00,128.655573,128.655573,128.655573,128.655573,2618.0
67,A,2021-04-08,2021-04-08 09:35:00,128.912884,129.971819,128.530381,129.942129,86243.0


### Calculate return in next x-minute

In [18]:
def calculate_return(df, window=RETURN_WINDOW):
    """
    Calculate percentage return in the next `window` 5-minute interval.
    """
    df = df.copy()
    df["datetime_next"] = df["datetime"] + pd.Timedelta(minutes=5*window)
    merged_df = df.merge(df[["symbol", "datetime", "close"]], 
                         left_on=["symbol", "datetime_next"], 
                         right_on=["symbol", "datetime"],
                         suffixes=("_before", "_after")
                        ).drop(columns="datetime_next")
    merged_df["pct_return"] = merged_df["close_after"]/merged_df["close_before"] - 1
    return merged_df

filled_stock_return_df = calculate_return(filled_stock_df).dropna()

In [19]:
assert (
    filled_stock_return_df["datetime_after"] - filled_stock_return_df["datetime_before"] 
        == 
    pd.Timedelta(minutes=5*RETURN_WINDOW)
       ).all()

In [20]:
filled_stock_return_df.head(10)

,symbol,date,datetime_before,open,high,low,close_before,volume,datetime_after,close_after,pct_return
0,A,2021-04-08,2021-04-08 08:35:00,128.655573,128.655573,128.655573,128.655573,700.0,2021-04-08 09:00:00,128.655573,0.000000
1,A,2021-04-08,2021-04-08 08:40:00,128.655573,128.655573,128.655573,128.655573,1642.0,2021-04-08 09:05:00,128.655573,0.000000
2,A,2021-04-08,2021-04-08 09:00:00,128.655573,128.655573,128.655573,128.655573,1642.0,2021-04-08 09:25:00,128.655573,0.000000
3,A,2021-04-08,2021-04-08 09:05:00,128.655573,128.655573,128.655573,128.655573,1642.0,2021-04-08 09:30:00,128.655573,0.000000
4,A,2021-04-08,2021-04-08 09:25:00,128.655573,128.655573,128.655573,128.655573,2618.0,2021-04-08 09:50:00,129.848111,0.009269
5,A,2021-04-08,2021-04-08 09:30:00,128.655573,128.655573,128.655573,128.655573,2618.0,2021-04-08 09:55:00,129.763990,0.008615
6,A,2021-04-08,2021-04-08 09:35:00,128.912884,129.971819,128.530381,129.942129,86243.0,2021-04-08 10:00:00,129.378024,-0.004341
7,A,2021-04-08,2021-04-08 09:40:00,129.942129,130.347889,129.793680,130.219233,26724.0,2021-04-08 10:05:00,129.308748,-0.006992
8,A,2021-04-08,2021-04-08 09:45:00,130.268716,130.367682,129.833266,129.892646,7491.0,2021-04-08 10:10:00,129.214730,-0.005219
9,A,2021-04-08,2021-04-08 09:50:00,129.922336,129.942129,129.773887,129.848111,6111.0,2021-04-08 10:15:00,129.140506,-0.005449


In [21]:
# create timezone columns
def create_timezone_cols(df):
    df = df.copy()
    df["datetime_us_eastern"] = df["datetime"].dt.tz_localize('US/Eastern')
    df["datetime_utc"] = df["datetime_us_eastern"].dt.tz_convert('UTC')
    df["dst"] = df["datetime_us_eastern"].apply(lambda x:x.dst())
    return df

# filled_stock_return_df = create_timezone_cols(filled_stock_return_df)

In [22]:
filled_stock_return_df

,symbol,date,datetime_before,open,high,low,close_before,volume,datetime_after,close_after,pct_return
0,A,2021-04-08,2021-04-08 08:35:00,128.655573,128.655573,128.655573,128.655573,700.0,2021-04-08 09:00:00,128.655573,0.000000
1,A,2021-04-08,2021-04-08 08:40:00,128.655573,128.655573,128.655573,128.655573,1642.0,2021-04-08 09:05:00,128.655573,0.000000
2,A,2021-04-08,2021-04-08 09:00:00,128.655573,128.655573,128.655573,128.655573,1642.0,2021-04-08 09:25:00,128.655573,0.000000
3,A,2021-04-08,2021-04-08 09:05:00,128.655573,128.655573,128.655573,128.655573,1642.0,2021-04-08 09:30:00,128.655573,0.000000
4,A,2021-04-08,2021-04-08 09:25:00,128.655573,128.655573,128.655573,128.655573,2618.0,2021-04-08 09:50:00,129.848111,0.009269
...,...,...,...,...,...,...,...,...,...,...,...
23826253,ZTS,2023-03-28,2023-03-28 15:45:00,164.775000,164.810000,164.660000,164.720000,16312.0,2023-03-28 16:10:00,165.040000,0.001943
23826254,ZTS,2023-03-28,2023-03-28 15:50:00,164.720000,164.750000,164.560000,164.635000,22097.0,2023-03-28 16:15:00,165.040000,0.002460
23826255,ZTS,2023-03-28,2023-03-28 15:55:00,164.735000,165.090000,164.690000,164.790000,33463.0,2023-03-28 16:20:00,165.040000,0.001517
23826256,ZTS,2023-03-28,2023-03-28 16:00:00,164.780000,165.070000,164.770000,165.050000,72463.0,2023-03-28 16:25:00,165.040000,-0.000061


## News

In [23]:
news_df = pd.read_csv("./news/news_v2/news_v2.csv", parse_dates=["time_published"])

In [24]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245719 entries, 0 to 245718
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   title                    245719 non-null  object        
 1   url                      245719 non-null  object        
 2   time_published           245719 non-null  datetime64[ns]
 3   authors                  245719 non-null  object        
 4   summary                  245705 non-null  object        
 5   banner_image             239106 non-null  object        
 6   source                   245719 non-null  object        
 7   category_within_source   72211 non-null   object        
 8   source_domain            245719 non-null  object        
 9   topics                   245719 non-null  object        
 10  overall_sentiment_score  245719 non-null  float64       
 11  overall_sentiment_label  245719 non-null  object        
 12  ticker_sentiment

In [25]:
news_df

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,company,symbol
0,The Charles Schwab Corporation ( SCHW ) Dips...,https://www.zacks.com/stock/news/1879310/the-c...,2022-03-08 23:00:11,['Zacks Investment Research'],The Charles Schwab Corporation (SCHW) closed a...,https://staticx-tuner.zacks.com/images/default...,Zacks Commentary,NaN,www.zacks.com,"[{'topic': 'Earnings', 'relevance_score': '0.9...",-0.048357,Neutral,"[{'ticker': 'SCHW', 'relevance_score': '0.5732...",Charles Schwab Corp,SCHW
1,The Charles Schwab Corporation ( SCHW ) Stoc...,https://www.zacks.com/stock/news/1882525/the-c...,2022-03-15 22:00:18,['Zacks Investment Research'],The Charles Schwab Corporation (SCHW) closed t...,https://staticx-tuner.zacks.com/images/default...,Zacks Commentary,NaN,www.zacks.com,"[{'topic': 'Earnings', 'relevance_score': '0.9...",0.018488,Neutral,"[{'ticker': 'SCHW', 'relevance_score': '0.5150...",Charles Schwab Corp,SCHW
2,Charles Schwab Joins Elite Club Of Stocks With...,https://www.investors.com/ibd-data-stories/cha...,2022-03-15 08:00:00,"[""INVESTOR'S BUSINESS DAILY"", ""Investor's Busi...",Charles Schwab Joins Elite Club Of Stocks With...,https://www.investors.com/wp-content/uploads/2...,Investors Business Daily,NaN,www.investors.com,"[{'topic': 'Earnings', 'relevance_score': '0.3...",0.039536,Neutral,"[{'ticker': 'HLI', 'relevance_score': '0.36101...",Charles Schwab Corp,SCHW
3,Schwab ( SCHW ) February Client Assets Fall ...,https://www.zacks.com/stock/news/1882042/schwa...,2022-03-15 12:22:00,['Zacks Investment Research'],Unfavorable markets hurt Schwab's (SCHW) total...,https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,NaN,www.zacks.com,"[{'topic': 'Earnings', 'relevance_score': '0.3...",-0.024247,Neutral,"[{'ticker': 'SCHW', 'relevance_score': '0.1565...",Charles Schwab Corp,SCHW
4,Should Schwab Fundamental U.S. Small Company I...,https://www.zacks.com/stock/news/1879406/shoul...,2022-03-09 11:20:06,['Zacks Investment Research'],Style Box ETF report for FNDA,https://staticx-tuner.zacks.com/images/default...,Zacks Commentary,NaN,www.zacks.com,"[{'topic': 'Retail & Wholesale', 'relevance_sc...",0.016141,Neutral,"[{'ticker': 'SCHW', 'relevance_score': '0.0613...",Charles Schwab Corp,SCHW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245714,T-Mobile US Unusual Options Activity For March...,https://www.benzinga.com/markets/options/23/03...,2023-03-30 20:01:07,['Benzinga Insights'],Someone with a lot of money to spend has taken...,https://www.benzinga.com/next-assets/images/sc...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.1...",0.117790,Neutral,"[{'ticker': 'TMUS', 'relevance_score': '0.7675...",T-Mobile US Inc.,TMUS
245715,"Arista ( ANET ) Offers Improved, Modernized ...",https://www.zacks.com/stock/news/2069120/arist...,2023-03-22 14:11:00,['Zacks Investment Research'],The Arista (ANET) solution will enable custome...,https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,NaN,www.zacks.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.280569,Somewhat-Bullish,"[{'ticker': 'TMUS', 'relevance_score': '0.0556...",T-Mobile US Inc.,TMUS
245716,T-Mobile ( TMUS ) Solution to Aid Roadside S...,https://www.zacks.com/stock/news/2069242/t-mob...,2023-03-22 17:09:00,['Zacks Investment Research'],T-Mobile (TMUS) will offer its wireless networ...,https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,NaN,www.zacks.com,"[{'topic': 'Earnings', 'relevance_score': '0.6...",0.323618,Somewhat-Bullish,"[{'ticker': 'TMUS', 'relevance_score': '0.1323...",T-Mobile US Inc.,TMUS
245717,5G IoT Market worth $59.7 billion by 2028 - Ex...,https://www.prnewswire.com/news-releases/5g-io...,2023-03-24 14:30:00,['MarketsandMarkets'],5G IoT Market worth $59.7 billion by 2028 - Ex...,https://mma.prnewswire.com/media/660509/Market...,PR Newswire

In [26]:
# check number of symbols
len(news_df["symbol"].unique())

480

In [27]:
news_df.groupby(["symbol"]).agg({"time_published":["min", "max", "count"]})

time_published                          
                       min                 max count
symbol                                              
A      2022-03-11 03:00:00 2023-03-31 05:54:42   358
AAL    2022-03-06 08:00:00 2023-03-31 13:00:00  1212
AAP    2022-03-04 13:17:00 2023-03-30 15:30:33   334
AAPL   2022-03-03 08:00:00 2023-03-30 16:38:00  4695
ABBV   2022-03-04 08:00:00 2023-03-30 11:45:00  1396
...                    ...                 ...   ...
YUM    2022-03-07 17:16:21 2023-03-30 15:11:25   290
ZBH    2022-03-09 16:30:18 2023-03-28 15:40:12   319
ZBRA   2022-03-07 16:00:00 2023-03-31 07:00:00   148
ZION   2022-07-21 13:54:00 2023-03-11 22:46:49    10
ZTS    2022-03-10 13:00:30 2023-03-17 22:15:24   157

[480 rows x 3 columns]

In [28]:
# round up to 5-minute level
news_df["time_published_5min"] = news_df["time_published"].dt.floor(freq="5T")

In [29]:
news_df[["symbol", "time_published", "time_published_5min"]]

,symbol,time_published,time_published_5min
0,SCHW,2022-03-08 23:00:11,2022-03-08 23:00:00
1,SCHW,2022-03-15 22:00:18,2022-03-15 22:00:00
2,SCHW,2022-03-15 08:00:00,2022-03-15 08:00:00
3,SCHW,2022-03-15 12:22:00,2022-03-15 12:20:00
4,SCHW,2022-03-09 11:20:06,2022-03-09 11:20:00
...,...,...,...
245714,TMUS,2023-03-30 20:01:07,2023-03-30 20:00:00
245715,TMUS,2023-03-22 14:11:00,2023-03-22 14:10:00
245716,TMUS,2023-03-22 17:09:00,2023-03-22 17:05:00
245717,TMUS,2023-03-24 14:30:00,2023-03-24 14:30:00


## Stock price - News 

In [30]:
news_price_df = news_df.merge(filled_stock_return_df, 
                              how="left", 
                              left_on=["symbol", "time_published_5min"],
                              right_on=["symbol", "datetime_before"],
                              validate="many_to_one")

In [31]:
filtered_news_price_df = news_price_df.dropna(subset=["pct_return"])
filtered_pct = len(filtered_news_price_df) / len(news_price_df)
print(f"Filter percentage {filtered_pct:.2%}")
print(f"Sample size before filtering: {len(news_df):,}")
print(f"Sample size after filtering: {len(filtered_news_price_df):,}")

Filter percentage 59.29%
Sample size after filtering: 245,719
Sample size after filtering: 145,689


In [32]:
filtered_news_price_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145689 entries, 3 to 245718
Data columns (total 26 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   title                    145689 non-null  object        
 1   url                      145689 non-null  object        
 2   time_published           145689 non-null  datetime64[ns]
 3   authors                  145689 non-null  object        
 4   summary                  145677 non-null  object        
 5   banner_image             141418 non-null  object        
 6   source                   145689 non-null  object        
 7   category_within_source   46206 non-null   object        
 8   source_domain            145689 non-null  object        
 9   topics                   145689 non-null  object        
 10  overall_sentiment_score  145689 non-null  float64       
 11  overall_sentiment_label  145689 non-null  object        
 12  ticker_sentiment

In [33]:
(filtered_news_price_df["pct_return"]*100).describe()

count    145689.000000
mean          0.005718
std           0.518308
min         -19.820467
25%          -0.186661
50%           0.000000
75%           0.194628
max          28.953580
Name: pct_return, dtype: float64

In [34]:
filtered_news_price_df.groupby("symbol").agg({"datetime_before":["min", "max", "count"]})

datetime_before                          
                       min                 max count
symbol                                              
A      2022-03-11 14:30:00 2023-03-27 13:45:00   178
AAL    2022-03-07 14:05:00 2023-03-29 17:05:00   941
AAP    2022-03-04 13:15:00 2023-03-13 15:10:00   169
AAPL   2022-03-03 08:00:00 2023-03-28 19:15:00  3384
ABBV   2022-03-04 08:00:00 2023-03-28 14:40:00   690
...                    ...                 ...   ...
YUM    2022-03-09 15:05:00 2023-03-28 13:45:00   176
ZBH    2022-03-09 16:30:00 2023-03-28 15:40:00    91
ZBRA   2022-03-07 16:00:00 2023-03-22 13:30:00    83
ZION   2022-07-21 13:50:00 2023-01-24 14:20:00     7
ZTS    2022-03-10 13:00:00 2023-03-16 15:30:00    86

[480 rows x 3 columns]

In [36]:
# filtered_news_price_df.to_csv("news_stock_price.csv", index=False)